In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import csv
from analysis import *

In [5]:
ripple_graph_base, caida_graph_base, _ = build_graphs('../Ripple/gateway_links.csv', '../Caida/20190301.as-rel2.txt')

/Users/yoanmartin/Google Drive/EPFL/Master II/Semester Project/student_19_ripple-partition/Gephi/analysis.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  links['link'] = list(zip(links['provider_peer'], links['customer_peer']))


In [101]:
locations = pd.read_csv('locations.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

In [102]:
locations['asn'] = locations[0].str.split('|').str[0]
locations['org'] = locations[0].str.split('|').str[3]


In [103]:
asn_org = locations[['asn','org']].set_index('asn').to_dict()['org']

In [104]:
organisations = pd.read_csv('organisations.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

In [105]:
organisations['org'] = organisations[0].str.split('|').str[0]
organisations['country'] = organisations[0].str.split('|').str[3]

In [106]:
org_country = organisations[['org','country']].set_index('org').to_dict()['country']

In [107]:
asn_country = {}
for n in ripple_graph_base.nodes:
    organisation = asn_org[n]
    country = org_country[organisation]
    asn_country[n] = country

In [108]:
countries_data = pd.read_csv('../Ripple/country.csv',delimiter=',', comment='#', encoding='ISO-8859-1')

In [109]:
countries = countries_data[['ISO 3166 Country Code', 'Latitude', 'Longitude']]

In [110]:
countries['lat-lon'] = list(zip(countries.Latitude,countries.Longitude))

In [111]:
countries_dict = countries[['ISO 3166 Country Code', 'lat-lon']].set_index('ISO 3166 Country Code').to_dict()['lat-lon']

In [112]:
asn_lat = {}
asn_lon = {}
for key,value in asn_country.items():
    asn_lat[key] = countries_dict[value][0]
    asn_lon[key] = countries_dict[value][1]

In [113]:
nx.set_node_attributes(ripple_graph_base, name='lat', values=asn_lat)
nx.set_node_attributes(ripple_graph_base, name='lon', values=asn_lon)

In [114]:
nx.write_gexf(ripple_graph_base,'ripple.gexf')